In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time

/Users/isabellawu/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [ ]:


# 爬取链接网页
def get_submission_data(driver, link):
    # 打开链接网页并切换到新窗口
    driver.execute_script("window.open();")
    driver.switch_to.window(driver.window_handles[1])
    driver.get(link)

    # 显示等待，等待页面中的特定元素加载完成
    time.sleep(6)

    # 使用Beautiful Soup解析页面源代码
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')

    # 定义一个辅助函数来提取字段值，处理可能的缺失字段
    def extract_field(field_name):
        field = soup.find('span', string=field_name)
        if field:
            return field.find_next('div', class_='slds-form-element__static').text
        return None

    # 提取链接对应页面的数据
    submission_data = {
        #'Submission ID': extract_field('Submission ID'),
        'Are you a third party, such as a law firm, trade association, or customs broker, submitting on behalf of an organization or industry? (OPTIONAL)': extract_field('Are you a third party, such as a law firm, trade association, or customs broker, submitting on behalf of an organization or industry? (OPTIONAL)'),
        '[2] Please select the COVID-related exclusion set to expire on September 30, 2021 related to your comment:': extract_field('[2] Please select the COVID-related exclusion set to expire on September 30, 2021 related to your comment:'),
        '[3] Do you support extending the exclusion beyond September 30, 2021 (yes or no)?': extract_field('[3] Do you support extending the exclusion beyond September 30, 2021 (yes or no)?'),
        '[4] Please provide your rationale for supporting or opposing an extension.': extract_field('[4] Please provide your rationale for supporting or opposing an extension.'),
        'Third Party Organizational Type (OPTIONAL)': extract_field('Third Party Organizational Type (OPTIONAL)'),
        'Third Party Firm, Association Name (OPTIONAL)': extract_field('Third Party Firm, Association Name (OPTIONAL)'),
        'Third Party Representative (REQUIRED)': extract_field('Third Party Representative (REQUIRED)')
    }
    #print(submission_data)

    # 关闭链接网页并切换回原窗口
    driver.close()
    driver.switch_to.window(driver.window_handles[0])

    return submission_data

# 爬取第一层网页并存储数据
def scrape_ustr_comments(url):
    driver = webdriver.Chrome()
    driver.get(url)

    data = {
        'Submission ID': [],
        'Commenter First Name': [],
        'Commenter Last Name': [],
        'Organization Name': [],
        'Annex List': [],
        'Link':[],
        'Are you a third party, such as a law firm, trade association, or customs broker, submitting on behalf of an organization or industry? (OPTIONAL)': [],
        '[2] Please select the COVID-related exclusion set to expire on September 30, 2021 related to your comment:': [],
        '[3] Do you support extending the exclusion beyond September 30, 2021 (yes or no)?': [],
        '[4] Please provide your rationale for supporting or opposing an extension.': [],
        'Third Party Organizational Type (OPTIONAL)': [],
        'Third Party Firm, Association Name (OPTIONAL)': [],
        'Third Party Representative (REQUIRED)': [],
    }

    while True:
        time.sleep(5)
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')

        tbody = soup.find('table').find('tbody') if soup.find('table') else None
        if tbody:
            for row in tbody.find_all('tr'):
                submission_id = row.find('a', href=True).text
                commenter_first_name = row.find_all('td')[0].text
                commenter_last_name = row.find_all('td')[1].text
                organization_name = row.find_all('td')[2].text
                annex_list = row.find_all('td')[3].text
                link = f"https://comments.ustr.gov{row.find('a', href=True)['href']}"

                submission_data = get_submission_data(driver, link)

                data['Submission ID'].append(submission_id)
                data['Commenter First Name'].append(commenter_first_name)
                data['Commenter Last Name'].append(commenter_last_name)
                data['Organization Name'].append(organization_name)
                data['Annex List'].append(annex_list)
                data['Link'].append(link)
                
                #print(submission_id)
                #print(organization_name)

                for field, value in submission_data.items():
                    data[field].append(value)
        
        # 翻页
        next_button = driver.find_element("xpath", "//button[contains(@class, 'slds-button_neutral') and contains(text(), 'Next')]")
        
        if "aria-disabled=\"true\"" in next_button.get_attribute("outerHTML"):
            break

        next_button.click()

    driver.quit()

    df = pd.DataFrame(data)
    df.to_excel('USTR-2021-0010.xlsx', index=False)

if __name__ == "__main__":
    url = 'https://comments.ustr.gov/s/docket?docketNumber=USTR-2021-0010'
    scrape_ustr_comments(url)


In [ ]:


# 爬取链接网页
def get_submission_data(driver, link):
    # 打开链接网页并切换到新窗口
    driver.execute_script("window.open();")
    driver.switch_to.window(driver.window_handles[1])
    driver.get(link)

    # 显示等待，等待页面中的特定元素加载完成
    time.sleep(3)

    # 使用Beautiful Soup解析页面源代码
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')

    # 定义一个辅助函数来提取字段值，处理可能的缺失字段
    def extract_field(field_name):
        field = soup.find('span', string=field_name)
        if field:
            return field.find_next('div', class_='slds-form-element__static').text
        return None

    # 提取链接对应页面的数据
    submission_data = {
        #'Submission ID': extract_field('Submission ID'),
        'Are you a third party, such as a law firm, trade association, or customs broker, submitting on behalf of an organization or industry? (OPTIONAL)': extract_field('Are you a third party, such as a law firm, trade association, or customs broker, submitting on behalf of an organization or industry? (OPTIONAL)'),
        '[1] For the COVID-related exclusion, please select the List(Annex) associated with the covered product.': extract_field('[1] For the COVID-related exclusion, please select the List(Annex) associated with the covered product.'),
        '[2] Please select the COVID-related exclusion you wish to comment on.': extract_field('[2] Please select the COVID-related exclusion you wish to comment on.'),
        '[3] Do you support extending the exclusion beyond May 15, 2023 (yes or no)?': extract_field('[3] Do you support extending the exclusion beyond May 15, 2023 (yes or no)?'),
        '[4] Please provide your rationale for supporting or opposing an extension.': extract_field('[4] Please provide your rationale for supporting or opposing an extension.'),
        '[5] Please explain whether the products covered by the exclusion are available from sources in the United States or third countries? (Please include information concerning changes in the global supply chain since 2020 with respect to the particular product or any other relevant industry developments).': extract_field('[5] Please explain whether the products covered by the exclusion are available from sources in the United States or third countries? (Please include information concerning changes in the global supply chain since 2020 with respect to the particular product or any other relevant industry developments).'),
        '[6] Please comment on whether extending or not extending the exclusion will impact the domestic supply of products covered by the exclusion, including the price and availability of the product.': extract_field('[6] Please comment on whether extending or not extending the exclusion will impact the domestic supply of products covered by the exclusion, including the price and availability of the product.')
    }
    #print(submission_data)

    # 关闭链接网页并切换回原窗口
    driver.close()
    driver.switch_to.window(driver.window_handles[0])

    return submission_data

# 爬取第一层网页并存储数据
def scrape_ustr_comments(url):
    driver = webdriver.Chrome()
    driver.get(url)

    data = {
        'Submission ID': [],
        'Commenter First Name': [],
        'Commenter Last Name': [],
        'Organization Name': [],
        'Annex List': [],
        'Link':[],
        'Are you a third party, such as a law firm, trade association, or customs broker, submitting on behalf of an organization or industry? (OPTIONAL)': [],
        '[1] For the COVID-related exclusion, please select the List(Annex) associated with the covered product.': [],
        '[2] Please select the COVID-related exclusion you wish to comment on.': [],
        '[3] Do you support extending the exclusion beyond May 15, 2023 (yes or no)?': [],
        '[4] Please provide your rationale for supporting or opposing an extension.': [],
        '[5] Please explain whether the products covered by the exclusion are available from sources in the United States or third countries? (Please include information concerning changes in the global supply chain since 2020 with respect to the particular product or any other relevant industry developments).': [],
        '[6] Please comment on whether extending or not extending the exclusion will impact the domestic supply of products covered by the exclusion, including the price and availability of the product.': [],
    }

    while True:
        time.sleep(3)
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')

        tbody = soup.find('table').find('tbody') if soup.find('table') else None
        if tbody:
            for row in tbody.find_all('tr'):
                submission_id = row.find('a', href=True).text
                commenter_first_name = row.find_all('td')[0].text
                commenter_last_name = row.find_all('td')[1].text
                organization_name = row.find_all('td')[2].text
                annex_list = row.find_all('td')[3].text
                link = f"https://comments.ustr.gov{row.find('a', href=True)['href']}"

                submission_data = get_submission_data(driver, link)

                data['Submission ID'].append(submission_id)
                data['Commenter First Name'].append(commenter_first_name)
                data['Commenter Last Name'].append(commenter_last_name)
                data['Organization Name'].append(organization_name)
                data['Annex List'].append(annex_list)
                data['Link'].append(link)
                
                #print(submission_id)
                #print(organization_name)

                for field, value in submission_data.items():
                    data[field].append(value)
        
        # 翻页
        next_button = driver.find_element("xpath", "//button[contains(@class, 'slds-button_neutral') and contains(text(), 'Next')]")
        
        if "aria-disabled=\"true\"" in next_button.get_attribute("outerHTML"):
            break

        next_button.click()

    driver.quit()

    df = pd.DataFrame(data)
    df.to_excel('USTR-2023-0001.xlsx', index=False)

if __name__ == "__main__":
    url = 'https://comments.ustr.gov/s/docket?docketNumber=USTR-2023-0001'
    scrape_ustr_comments(url)
